# top

# top

In [1]:
import os, sys
import pickle
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
import sklearn.metrics
import sklearn.ensemble

sys.path.append('..')
import diefComp1Utils as util


/Users/arianprabowo/anaconda3/envs/dief3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folderPath = '/Users/arianprabowo/Library/CloudStorage/OneDrive-UNSW/root/DIEF/raw data/e04_splits/'
pathTrainX = folderPath+'train_X.zip'
pathTrainY = folderPath+'train_Y.csv'
pathTestX = folderPath+'test_X.zip'
pathTestY = folderPath+'test_Y.csv'

# load

In [3]:
dfTrainY = pd.read_csv(pathTrainY, index_col=0)
aTrny = dfTrainY.iloc[:,3:].values ==1

In [4]:
aTrnx = util.load_GlobalFeatureSet1(pathTrainX)

  0%|          | 0/31851 [00:00<?, ?it/s]/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:164: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,2] = scipy.stats.skew(idata['v'])
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:165: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,3] = scipy.stats.kurtosis(idata['v'])
100%|██████████| 31851/31851 [00:30<00:00, 1035.68it/s]


In [5]:
dfTestY = pd.read_csv(pathTestY, index_col=0)
aTsty = dfTestY.iloc[:,3:].values ==1

In [6]:
aTstx = util.load_GlobalFeatureSet1(pathTestX)

  0%|          | 0/329439 [00:00<?, ?it/s]/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:164: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,2] = scipy.stats.skew(idata['v'])
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:165: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  adata[ii,3] = scipy.stats.kurtosis(idata['v'])
100%|██████████| 329439/329439 [04:52<00:00, 1125.65it/s]


# 10x replications

In [7]:
def run_rf(rs):
    m = sklearn.ensemble.RandomForestClassifier(random_state=rs)
    m.fit(aTrnx, aTrny) # 3 mins
    aTrnh = m.predict(aTrnx)
    aTsth = m.predict(aTstx) # 10 mins
    return aTrnh.astype(float), aTsth.astype(float)

In [8]:
n = 10
z = np.empty((4,n,10))
for i in tqdm(range(n)):
    trn_H, tst_H = run_rf(i)
    d = util.allMetrics(aTrny, trn_H, verbose=False)
    z[0,i,0] = d['accuracy'].mean()
    z[0,i,1] = d['precision'].mean()
    z[0,i,2] = d['recall'].mean()
    z[0,i,3] = d['f1'].mean()
    z[0,i,4] = d['AP'].mean()
    dfM, dfWM = util.weighted_metrics(d, dfTrainY)
    z[0,i,5:] = dfWM.iloc[:]
    # d = util.parition_wrapper_for_metrics(dfTestY, tst_H, 'leaderboard', verbose=False)
    # z[1,i,0] = d['accuracy'].mean()
    # z[1,i,1] = d['precision'].mean()
    # z[1,i,2] = d['recall'].mean()
    # z[1,i,3] = d['f1'].mean()
    # z[1,i,4] = d['AP'].mean()
    # d = util.parition_wrapper_for_metrics(dfTestY, tst_H, 'secret', verbose=False)
    # z[2,i,0] = d['accuracy'].mean()
    # z[2,i,1] = d['precision'].mean()
    # z[2,i,2] = d['recall'].mean()
    # z[2,i,3] = d['f1'].mean()
    # z[2,i,4] = d['AP'].mean()
    d = util.parition_wrapper_for_metrics(dfTestY, tst_H, 'combined', verbose=False)
    z[3,i,0] = d['accuracy'].mean()
    z[3,i,1] = d['precision'].mean()
    z[3,i,2] = d['recall'].mean()
    z[3,i,3] = d['f1'].mean()
    z[3,i,4] = d['AP'].mean()
    dfM, dfWM = util.weighted_metrics(d, dfTestY)
    z[0,i,5:] = dfWM.iloc[:]
u = z.mean(axis=1)
s = z.std(axis=1)
for i, ii in enumerate(['train', 'leader', 'secret', 'combined']):
    if i==1 or i==2:
        continue
    for j, jj in enumerate(['accuracy', 'precision', 'recall', 'f1', 'AP']):
        print(ii, jj, 'mean,', u[i,5+j])
        print(ii, jj, 'std ,', s[i,5+j])

  0%|          | 0/10 [00:00<?, ?it/s]/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:306: RuntimeWarning: invalid value encountered in divide
  accuracy = (tp + tn) / (tp + tn + fp + fn)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:307: RuntimeWarning: invalid value encountered in divide
  precision = tp / (tp + fp)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:308: RuntimeWarning: invalid value encountered in divide
  recall = tp / (tp + fn)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:263: RuntimeWarning: invalid value encountered in divide
  a_precision[ii,:] = true__positive / (true__positive + false_positive)
/Users/arianprabowo/My Drive/employment/2023 09 UNSW postdoc/DIEF/e08_RF/../diefComp1Utils.py:264: RuntimeWarning: invalid value encountered in divide
  a_recall[ii,:] = true__positive 

train accuracy mean, 0.8505277093548251
train accuracy std , 0.00432778586669817
train precision mean, 0.0856330238666839
train precision std , 0.07439935133357219
train recall mean, 0.03745725591002195
train recall std , 0.07489463165711946
train f1 mean, 0.02592612301741127
train f1 std , 0.05181249515619223
train AP mean, 0.05272337294071362
train AP std , 0.0436926023162738
combined accuracy mean, 2.2906148337e-314
combined accuracy std , 0.0
combined precision mean, 2.204010727e-314
combined precision std , 0.0
combined recall mean, 2.373714695e-314
combined recall std , 0.0
combined f1 mean, 2.296042447e-314
combined f1 std , 0.0
combined AP mean, 2.148449853e-314
combined AP std , 0.0


[11:55<1:47:19, 715.55s/it]

100%|██████████| 10/10 [2:00:57<00:00, 725.75s/it]

train accuracy mean, 0.8505277093548251
train accuracy std , 0.00432778586669817
train precision mean, 0.0856330238666839
train precision std , 0.07439935133357219
train recall mean, 0.03745725591002195
train recall std , 0.07489463165711946
train f1 mean, 0.02592612301741127
train f1 std , 0.05181249515619223
train AP mean, 0.05272337294071362
train AP std , 0.0436926023162738
combined accuracy mean, 2.2906148337e-314
combined accuracy std , 0.0
combined precision mean, 2.204010727e-314
combined precision std , 0.0
combined recall mean, 2.373714695e-314
combined recall std , 0.0
combined f1 mean, 2.296042447e-314
combined f1 std , 0.0
combined AP mean, 2.148449853e-314
combined AP std , 0.0

In [10]:
u = z.mean(axis=1)
s = z.std(axis=1)
for i, ii in enumerate(['train', 'leader', 'secret', 'combined']):
    if i==1 or i==2:
        continue
    for j, jj in enumerate(['accuracy', 'precision', 'recall', 'f1', 'AP', 'Waccuracy', 'Wprecision', 'Wrecall', 'Wf1', 'wAP']):
        print(ii, jj, 'mean,', u[i,j])
        print(ii, jj, 'std ,', s[i,j])

train accuracy mean, 0.42227164997911715
train accuracy std , 0.0010257239419125606
train precision mean, 0.7583333333333332
train precision std , 1.1102230246251565e-16
train recall mean, 0.42227164997911715
train recall std , 0.0010257239419125606
train f1 mean, 0.5389406612783574
train f1 std , 0.00094819853445206
train AP mean, 0.5903024916562252
train AP std , 0.0005128619709562609
train Waccuracy mean, 0.8505277093548251
train Waccuracy std , 0.00432778586669817
train Wprecision mean, 0.0856330238666839
train Wprecision std , 0.07439935133357219
train Wrecall mean, 0.03745725591002195
train Wrecall std , 0.07489463165711946
train Wf1 mean, 0.02592612301741127
train Wf1 std , 0.05181249515619223
train wAP mean, 0.05272337294071362
train wAP std , 0.0436926023162738
combined accuracy mean, 0.9881935728457243
combined accuracy std , 9.630265945887751e-05
combined precision mean, 0.0019055212503289575
combined precision std , 0.0016555475747011225
combined recall mean, 0.000833505508

# bot

In [9]:
z[0,:,0]

array([0.42321285, 0.42223746, 0.42265259, 0.42227853, 0.42084592,
       0.4220429 , 0.42312958, 0.42304229, 0.4200141 , 0.42326028])